In [11]:
# !pip install langchain unstructured python-docx
!pip install python-docx


In [6]:
import pandas as pd
import time
import json, os
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
from langchain_chroma import Chroma
from langchain.document_loaders import UnstructuredFileLoader
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langfuse.callback import CallbackHandler
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [2]:
KVUri = "https://dipanjans-kv.vault.azure.net"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)


oai_endpoint_embed= client.get_secret("azure-openai-endpoint-embed")
oai_key_embed=      client.get_secret("azure-openai-key-embed")
oai_endpoint=       client.get_secret("azure-openai-endpoint")
oai_key=      client.get_secret("azure-openai-key")

embeds = AzureOpenAIEmbeddings(azure_deployment="dipanjan_ada_embed_150k",
                        model = 'text-embedding-ada-002',
                        azure_endpoint= oai_endpoint_embed.value,
                        api_key = oai_key_embed.value)

llm = AzureChatOpenAI(api_version="2024-12-01-preview",
                        model='gpt-40-mini' , 
                        azure_endpoint= oai_endpoint.value,
                        api_key = oai_key.value)

####### Testing llm and embed model ######
llm.invoke("who is this?")
# print(out[0])
# print("****")
# print(out[1])

AIMessage(content="Could you please provide more context or specify whom you are referring to? Without additional information, I'm not able to identify anyone.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_57db37749c', 'id': 'chatcmpl-BiHN5wSOiPgST1krh2JvuNKSPkEIy', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_r

In [3]:
# transcript_1 = """Transcript: Software Engineering Team Standup – April 11, 2025

# [09:02 AM – Standup Begins]

# Anna:
# Good morning, everyone. Let's keep this quick but thorough. We'll go team member by team member and then do a short backlog grooming. Please mention blockers, new bugs, and any action items. Jack, you're up first.

# Jack:
# Yeah, so I finished the API rate-limiting middleware yesterday and deployed it to staging. QA found one bug – if a token is missing, it throws a 500 instead of a 401. I'll fix that today.
# Also, I was supposed to start on the user analytics dashboard backend, but we realized the schema for event_logs isn't finalized. I'll talk to Sam after the meeting. No blockers otherwise.

# Anna:
# Got it. Bug sounds urgent. Please raise a ticket. And let's Slack Sam to finalize the schema – I'll do that.
# Next, Maria?

# Maria:
# Thanks. I wrapped up the design updates for the login page, pushed to feature/login-redesign. There's an open PR.
# I noticed that the forgot-password page doesn't have a responsive layout – I'll fix that today.
# Also, we had a mix-up. The new feature flag toggle UI task was unassigned but marked in sprint planning. I think it was meant for me?

# Anna:
# Yes, thanks for catching that. Can you take it?

# Maria:
# Yep, I'll own it.

# Anna:
# Great. Sam?

# Sam:
# Morning! So, still working on cleaning the event logs for dashboard reporting. I've had to rewrite a few ETL jobs – there's a weird edge case where user_id is null for anonymous events. I'll fix it today and finalize the schema before lunch.
# Also, someone needs to inform the marketing team that their requested funnel metrics will be delayed – anyone want to take that?

# Anna:
# I'll Slack Aisha in marketing after this. Thanks for the heads up.

# David:
# Quick update from me – I got the biometric login working on Android. On iOS, Face ID works but Touch ID throws an auth error. I'll debug that today.
# I've also started refactoring the push notification service, as we discussed last week, but I might need help validating tokens server-side. Jack, maybe you can help?

# Jack:
# Sure. Ping me after lunch.

# Anna:
# Let's also note that as a follow-up action.
# Priya?

# Priya:
# Hey! Found a couple of bugs during regression testing on the staging app.
# 1.	The new login design sometimes overlays the spinner and the input field.
# 2.	On smaller screens, the sidebar navigation breaks.
# 3.	And the biometric login sometimes crashes on older Android versions.
# I've logged bugs 1 and 2 in Jira. Bug 3 I need to test a bit more – might be OS-specific.

# Anna:
# Perfect. Please assign the bugs to the respective devs when ready.
# Quick note: please remember we have a soft deadline for the public beta next Friday – April 18th.

# Jack:
# Wait – for the beta, do we also need the reporting dashboard?

# Anna:
# Ideally, yes – at least the MVP version.

# Sam:
# In that case, I might need help from someone else on transforming some of the data.

# Anna:
# Maybe we can split that task. David, any bandwidth end of week?

# David:
# Yeah, after Thursday. Assign me to any subtask.

# Anna:
# Awesome. Let's make that part of the plan.
# One last thing – the accessibility review hasn't been scheduled. We need someone to contact DevRel to get it booked.

# Maria:
# I can message Anika – she's on DevRel, right?

# Anna:
# Awesome. Let's do that.
# Okay, before we wrap up—just quickly—anything else we need to flag before retro on Friday?

# Jack:
# Actually, yeah. About the cache invalidation for the user profile updates… We've had a few reports that changes don't reflect immediately in the mobile app. It's not technically a bug, but it's a poor user experience.

# David:
# Oh right, I saw that too. I assumed it was expected behavior, but now that you mention it, it's kind of jarring.

# Jack:
# I could add a hook to purge the cache on successful update, but we'd need to be careful about race conditions. Maybe we should brainstorm a more solid fix.

# Anna:
# Let's flag this for retro and maybe spin off a tech debt ticket.
# David, Jack—can you two jot down the key problems you're seeing and propose options before then?

# David:
# Sure, we'll sync tomorrow.

# Maria:
# Also—sorry to add last-minute stuff—but while testing the new color palette, I noticed our dark mode support is broken on a couple of routes. Should I fix it or log a ticket?

# Anna:
# If you've got time, go for it. Otherwise, just raise it and we'll triage.

# Maria:
# I can squeeze it in today.

# Priya:
# Heads up though: please test on both Chrome and Firefox. There were some rendering issues last time.

# Maria:
# Will do. Thanks.

# Sam:
# Oh, and before I forget—I had a conversation with James in Legal last week. He mentioned we need to audit user data retention in the analytics pipeline. It's not urgent, but we probably need a plan by the end of the month.

# Anna:
# That's... yeah. A big one. Can you write a doc with the current flow and mark any areas we're unsure about?

# Sam:
# Yep. I'll try to get that done by Thursday.

# Anna:
# Thanks. Also, if there's anything sensitive or needs redaction, flag it.
# Final call—anything else?

# Jack:
# Actually, wait. Remember the auto-logout feature we planned? Should we still do it this sprint? It hasn't been assigned yet.

# Anna:
# Ugh, we forgot again. Anyone have room to pick it up?

# Maria:
# I can maybe take it if I postpone the toggle UI task to next sprint?

# Anna:
# That's fine with me. Toggle's not launch-critical. Let's do that. Reassigning now.

# David:
# Also, can we remind DevOps to bump staging to Node 18? I think we're still on 16.

# Anna:
# Good call. I'll Slack Ravi after this.

# Priya:
# Wait—if we update Node, does that break the older build tools?

# Jack:
# Possibly. I'll run a test build and report back.

# Anna:
# Okay team, that's all from me. Reminder:
# 1.	Retro is Friday at 2 PM.
# 2.	Soft beta deadline is April 18th.
# 3.	Please comment on the sprint board if anything changes before then.
# 4.	Let's get moving. Thanks, all!"""

# from docx import Document

# doc = Document()

# for line in transcript_1.split('\n'):
#     doc.add_paragraph(line)

# # Save the document
# doc.save("sample_meet_transcript.docx")



In [7]:
# Load the DOCX file. A sample transcript from kaggle 
# was converted to docx to mimic MS 
# teams meet transcript downloaded in docx format
def load_transcript(file):
    loader = UnstructuredFileLoader(file)
    documents = loader.load()
    
    transcript_str =''
    for doc in documents:
        transcript_str = transcript_str + doc.page_content
    return transcript_str

In [12]:
langfuse_sk= client.get_secret("langfuse-secret-key-3")
langfuse_pk= client.get_secret("langfuse-public-key-3")
langfuse_host= client.get_secret("langfuse-host-3")

langfuse_handler = CallbackHandler(
    secret_key= langfuse_sk.value,
    public_key=langfuse_pk.value,
    host=langfuse_host.value,
    session_id = 'dag_chain_finance_uuid'
)

In [ ]:
# Transcript summarizer and action item extractor
def summarizer_action_item_extract(transcript):
    class summary_action_item(BaseModel):
        summary: str = Field(description="Meeting summary in bullet points")
        action_items: str = Field(description="Meeting action items / work items")
    
    json_parser_summary_action_item = JsonOutputParser(pydantic_object= summary_action_item)
    
    
    message = """
            You are an expert meeting notes taker. Use the transcript given below to come up with a meeting summary.
            Use the transcript and also give the action items *in bullet points*.
            If there are no action items , say no action_items in the meeting.
            Do not make up the answer, do not provide answer from outside the given transcript.
            Adhere to the format instructions - {format_instructions}

            **Transcript**:
            {transcript}
            """
    prompt = PromptTemplate(template= message,
                            input_variables=["transcript"],
                            partial_variables={"format_instructions": json_parser_summary_action_item.get_format_instructions()})
    
    summarizer_chain = prompt | llm | json_parser_summary_action_item
    results= summarizer_chain.invoke(transcript, config={"callbacks": [langfuse_handler]})
    return results

dct = summarizer_action_item_extract(load_transcript("sample_meet_transcript.docx"))
print(dct['summary'], dct['action_items'])
    

• Jack completed the API rate-limiting middleware and deployed to staging. A bug was found that he will fix today. He is also waiting on schema finalization for user analytics dashboard. 
• Maria finished design updates for the login page and will fix the forgot-password page layout today. She will also take over the unassigned feature flag toggle UI task. 
• Sam is cleaning event logs for dashboard reporting and will finalize schema before lunch. He will inform the marketing team about the delayed funnel metrics. 
• David got biometric login working on Android and is debugging Touch ID on iOS. He started refactoring the push notification service and may need help validating tokens from Jack. 
• Priya found bugs during regression testing and logged some in Jira. She will assign them when ready. 
• The team discussed the need for the reporting dashboard by the public beta deadline and split tasks accordingly. 
• Maria will contact DevRel for accessibility review scheduling. 
• The team 